In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

In [ ]:
mode_number = 2**20
FSR = 25e9
omega_m = 2 * np.pi * FSR
xs = np.linspace(0, 1 / FSR, mode_number)
xs_freq = np.fft.fftshift(np.fft.fftfreq(mode_number, xs[1] - xs[0]))
modes = np.arange(-mode_number // 2, mode_number // 2)

gamma = 0
k = 0.03
alpha = 0.95
phi_o = 0
phi_m = 0
phi_d = 1e-3
r = np.sqrt(alpha * (1 - gamma) * (1 - k)) * np.exp(-1j * phi_o)
M = 0.3 * np.pi



In [ ]:
# Round trip model

E_intra = np.sqrt(k / (1 - k)) * (r * np.exp(1j * M * np.sin(omega_m * xs))) / (1 - r * np.exp(1j * M * np.sin(omega_m * xs)))
# E_intra = np.roll(E_intra, int(mode_number / 4))
# E_intra = np.roll(E_intra, int(mode_number / 2))
E_intra_spectrum = np.abs(np.fft.fftshift(np.fft.fft(E_intra)))**2 / mode_number**2
E_intra_phase = np.angle(np.fft.fftshift(np.fft.fft(E_intra)))
E_out = np.sqrt((1 - gamma) * (1 - k)) - np.sqrt(k * (1 - gamma)) * E_intra
E_out_spectrum = np.abs(np.fft.fftshift(np.fft.fft(E_out)))**2
E_out_phase = np.angle(np.fft.fftshift(np.fft.fft(E_out)))

In [ ]:
window = 2**6

plt.figure()
plt.plot(modes[mode_number//2-window:mode_number//2+window], E_intra_spectrum[mode_number//2-window:mode_number//2+window], label='E_intra_spectrum')
plt.yscale('log')
plt.show()

plt.figure()
plt.plot(modes[mode_number//2-window:mode_number//2+window], E_intra_phase[mode_number//2-window:mode_number//2+window], label='E_intra_phase')
plt.show()

plt.figure()
plt.plot(xs, np.abs(E_intra), label='E_intra')
plt.show()

In [ ]:
window = 2**6

plt.figure()
plt.plot(modes[mode_number//2-window:mode_number//2+window], E_out_spectrum[mode_number//2-window:mode_number//2+window], label='E_out_spectrum')
plt.yscale('log')
plt.show()

plt.figure()
plt.plot(modes[mode_number//2-window:mode_number//2+window], E_out_phase[mode_number//2-window:mode_number//2+window], label='E_out_phase')
plt.show()

plt.figure()
plt.plot(xs, np.abs(E_out), label='E_out')
plt.show()

In [ ]:
# Steady state matrix

def phi_total(p):
    phi_tot = phi_o + p * phi_m + p**2 * phi_d
    return phi_tot

mode_number = 2**10
xs = np.linspace(0, 1 / FSR, mode_number)
xs_freq = np.fft.fftshift(np.fft.fftfreq(mode_number, xs[1] - xs[0]))
modes = np.arange(-mode_number // 2, mode_number // 2)


matrix_operator = np.zeros((mode_number, mode_number), dtype=complex)
for p in modes:
    for q in modes:
        matrix_operator[p - modes[0], q - modes[0]] = r * sp.special.jv(p - q, M) * np.exp(1j * phi_total(q))

B = np.zeros(mode_number, dtype=complex)
for p in modes:
    B[p - modes[0]] = np.sqrt(k / (1 - k)) * sp.special.jv(p, M) * np.exp(1j * phi_total(0))

# print(matrix_operator)
# print(B)
# print(modes)




In [ ]:
solution = np.linalg.solve(np.eye(mode_number) - matrix_operator, B)

spectrum_steady = np.abs(solution)**2
phase_steady = np.angle(solution)
E_steady = np.fft.ifft(np.fft.ifftshift(solution)) * mode_number

In [ ]:
window = 100

plt.figure()
plt.plot(modes[mode_number//2-window:mode_number//2+window], spectrum_steady[mode_number//2-window:mode_number//2+window], label='spectrum_steady')
plt.yscale('log')
plt.show()

plt.figure()
plt.plot(modes[mode_number//2-window:mode_number//2+window],phase_steady[mode_number//2-window:mode_number//2+window], label='phase_steady')
plt.show()

plt.figure()
plt.plot(modes, np.abs(E_steady), label='E_steady')
plt.show()